In [1]:
import pandas as pd
import numpy as np

In [2]:
def GroupbyTime(df, time):
    countmin = 1*60*1000000000
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['datetime'] = pd.to_datetime((df['datetime'].view(np.int64) // countmin) * countmin) + pd.to_timedelta('8h')
    df = df.groupby('datetime').mean()
    df = pd.merge(time, df, left_on='timestamp', right_on='datetime', how='left')
    return df

In [3]:
col_rename = {
    'node_load5 192.168.31.5:9100': 'node_load_ip5',
    'node_load5 192.168.31.6:9100': 'node_load_ip6', 
    'node_load5 192.168.31.7:9100': 'node_load_ip7',
    'cpu_usage 192.168.31.5:9100': 'cpu_usage_ip5', 
    'cpu_usage 192.168.31.6:9100': 'cpu_usage_ip6',
    'cpu_usage 192.168.31.7:9100': 'cpu_usage_ip7',
    'mem_usage 192.168.31.5:9100': 'mem_usage_ip5',
    'mem_usage 192.168.31.6:9100': 'mem_usage_ip6',
    'mem_usage 192.168.31.7:9100': 'mem_usage_ip7',
    
    'cpu_usage carts-db-844bf68bff-k7fvm': 'cpu_usage_carts_db',
    'cpu_usage orders-db-9fd454cff-rrttb': 'cpu_usage_orders_db',
    'cpu_usage catalogue-db-5c6545dd69-lltrq': 'cpu_usage_catalogue_db',
    'cpu_usage user-db-64c449b57-m8llg': 'cpu_usage_user_db',
    'cpu_usage rabbitmq-7ddc84cb6c-9d9f6': 'cpu_usage_rabbitmq',
    'cpu_usage session-db-8f64655d5-qtnwh': 'cpu_usage_session_db',
    'cpu_usage orders-5ffb5f8596-d74jv': 'cpu_usage_orders', 
    'cpu_usage user-55b65b94bf-s98kg': 'cpu_usage_user',
    'cpu_usage queue-master-6bf566994f-hvrpm': 'cpu_usage_queue_master',
    'cpu_usage front-end-8648798447-tf4cj': 'cpu_usage_front_end',
    
    'cpu_usage payment-796848994-wwhv4': 'cpu_usage_payment', 
    'cpu_usage carts-64f7c6d9d7-5mmd6': 'cpu_usage_carts',
    'cpu_usage shipping-899f9897f-96484': 'cpu_usage_shipping',
    'cpu_usage catalogue-6d5b4c4c4b-tpl5w': 'cpu_usage_catalogue',
    'mem_usage carts-db-844bf68bff-k7fvm(MiB)': 'mem_usage_carts_db',
    'mem_usage orders-db-9fd454cff-rrttb(MiB)': 'mem_usage_orders_db',
    'mem_usage catalogue-db-5c6545dd69-lltrq(MiB)': 'mem_usage_catalogue_db',
    'mem_usage user-db-64c449b57-m8llg(MiB)': 'mem_usage_user_db',
    'mem_usage rabbitmq-7ddc84cb6c-9d9f6(MiB)': 'mem_usage_rabbitmq',
    'mem_usage session-db-8f64655d5-qtnwh(MiB)': 'mem_usage_session_db',
    
    'mem_usage orders-5ffb5f8596-d74jv(MiB)': 'mem_usage_orders',
    'mem_usage user-55b65b94bf-s98kg(MiB)': 'mem_usage_user',
    'mem_usage queue-master-6bf566994f-hvrpm(MiB)': 'mem_usage_queue_master',
    'mem_usage front-end-8648798447-tf4cj(MiB)': 'mem_usage_front_end',
    'mem_usage payment-796848994-wwhv4(MiB)': 'mem_usage_payment',
    'mem_usage carts-64f7c6d9d7-5mmd6(MiB)': 'mem_usage_carts',
    'mem_usage shipping-899f9897f-96484(MiB)': 'mem_usage_shipping',
    'mem_usage catalogue-6d5b4c4c4b-tpl5w(MiB)': 'mem_usage_catalogue', 
    'order_qps(ops/s)': 'order_qps',
    'order_latency(ms)': 'order_latency', 
    
    'catalogue_qps(ops/s)': 'catalogue_qps', 
    'catalogue_latency(ms)': 'catalogue_latency',
    'payment_qps(ops/s)': 'payment_qps', 
    'payment_latency(ms)': 'payment_latency', 
    'shipping_qps(ops/s)': 'shipping_qps',
    'shipping_latency(ms)': 'shipping_latency', 
    'user_qps(ops/s)': 'user_qps', 
    'user_latency(ms)': 'user_latency',
    'frontend_qps(ops/s)': 'frontend_qps', 
    'frontend_latency(ms)': 'frontend_latency', 
    
    'cart_qps(ops/s)': 'cart_qps',
    'cart_latency(ms)': 'cart_latency'
}

In [4]:
col = ['timestamp']+sorted(list(set(col_rename.values())))

# 训练集处理

In [5]:
time = pd.DataFrame(pd.date_range(start='2022-03-24 15:20', end='2022-03-25 15:19', freq='T', name='timestamp'))

In [6]:
train_node = GroupbyTime(pd.read_csv('../dataset/sockshop/train/metrics/node_metrics.csv'), time)
train_pod = GroupbyTime(pd.read_csv('../dataset/sockshop/train/metrics/pod_metrics.csv'), time)
train_service = GroupbyTime(pd.read_csv('../dataset/sockshop/train/metrics/service_metrics.csv'), time)

train = pd.merge(pd.merge(train_node, train_pod, on='timestamp'), train_service, on='timestamp')
train.rename(columns=col_rename, inplace=True)
train = train[col]
train.head()

,timestamp,cart_latency,cart_qps,catalogue_latency,catalogue_qps,cpu_usage_carts,cpu_usage_carts_db,cpu_usage_catalogue,cpu_usage_catalogue_db,cpu_usage_front_end,...,node_load_ip6,node_load_ip7,order_latency,order_qps,payment_latency,payment_qps,shipping_latency,shipping_qps,user_latency,user_qps
0,2022-03-24 15:20:00,0.001230,66.666667,0.000654,66.668148,0.002298,0.005971,0.004414,0.000811,0.023395,...,0.4000,1.3200,0.001457,66.666667,0.000543,66.666667,0.000313,66.666667,0.001150,66.666667
1,2022-03-24 15:21:00,0.001956,86.868167,0.011547,99.999494,0.003245,0.006133,0.004523,0.001022,0.023863,...,0.3500,1.3025,0.003039,66.665556,0.002402,67.778148,0.000328,66.666667,0.001764,106.665975
2,2022-03-24 15:22:00,0.002750,111.666667,0.001357,111.111790,0.004568,0.006534,0.004657,0.001094,0.025216,...,0.3325,1.2525,0.016204,75.118500,0.001446,69.443679,0.000383,68.888889,0.001583,133.334086
3,2022-03-24 15:23:00,0.002415,110.555556,0.001220,104.998778,0.005810,0.006856,0.004791,0.001161,0.027565,...,0.3650,1.2175,0.007996,75.555556,0.000543,70.000395,0.000324,69.444444,0.001847,147.775272
4,2022-03-24 15:24:00,0.002614,116.111111,0.001058,126.111111,0.007748,0.007269,0.004992,0.001266,0.030932,...,0.3450,1.1850,0.009291,76.111111,0.000543,69.444444,0.000304,70.000000,0.001601,147.221938


In [7]:
train.to_csv('../dataset/processed/train/metrics/metrics.csv', index=False)

# 测试集处理

In [8]:
time = pd.DataFrame(pd.date_range(start='2022-03-26 08:30', end='2022-03-26 20:29', freq='T', name='timestamp'))

In [9]:
test_node = GroupbyTime(pd.read_csv('../dataset/sockshop/test/metrics/node_metrics_abn.csv'), time)
test_pod = GroupbyTime(pd.read_csv('../dataset/sockshop/test/metrics/pod_metrics_abn.csv'), time)
test_service = GroupbyTime(pd.read_csv('../dataset/sockshop/test/metrics/service_metrics_abn.csv'), time)

test = pd.merge(pd.merge(test_node, test_pod, on='timestamp'), test_service, on='timestamp')
test.rename(columns=col_rename, inplace=True)
test = test[col]
test.head()

,timestamp,cart_latency,cart_qps,catalogue_latency,catalogue_qps,cpu_usage_carts,cpu_usage_carts_db,cpu_usage_catalogue,cpu_usage_catalogue_db,cpu_usage_front_end,...,node_load_ip6,node_load_ip7,order_latency,order_qps,payment_latency,payment_qps,shipping_latency,shipping_qps,user_latency,user_qps
0,2022-03-26 08:30:00,0.006822,157.037037,0.001513,295.555144,0.006199,0.008771,0.005756,0.001368,0.064068,...,0.7100,0.1425,0.002178,71.851852,0.000757,72.593663,0.000445,68.888889,0.001773,181.481235
1,2022-03-26 08:31:00,0.004399,137.777778,0.001233,148.889642,0.006497,0.009097,0.005816,0.001383,0.068141,...,0.6875,0.1525,0.003579,76.111111,0.000652,72.221802,0.000435,69.444444,0.001716,166.111901
2,2022-03-26 08:32:00,0.003314,144.444444,0.000911,152.220642,0.006520,0.009023,0.005803,0.001317,0.068913,...,0.6750,0.1375,0.004039,83.888889,0.000541,72.223037,0.000462,72.222222,0.001651,179.998148
3,2022-03-26 08:33:00,0.004319,155.000000,0.000821,201.670321,0.006822,0.009183,0.005945,0.001383,0.074047,...,0.9850,0.1275,0.003875,79.444444,0.000583,73.333333,0.000467,71.111111,0.001829,187.779136
4,2022-03-26 08:34:00,0.002712,86.666667,0.001123,89.443333,0.006184,0.008838,0.005620,0.001290,0.070186,...,0.9050,0.1950,0.001054,67.222222,0.000594,67.776642,0.000436,67.222222,0.001501,84.444975


In [10]:
test.to_csv('../dataset/processed/test/metrics/metrics.csv', index=False)